In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
pip install bertopic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 25.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.9/90.9 kB 12.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 20.3 MB/s eta 0:00:00
  Using cached Cython-0.29.37-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (1.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 38.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 49.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 47.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/73

In [4]:
# Import Libraries
import pandas as pd
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
import pandas as pd
from bertopic import BERTopic
import matplotlib.pyplot as plt
import gensim
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import string
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('vader_lexicon')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [5]:
# Import Self Classified (Roberta Classifier : 1) tweets dataset
df = pd.read_csv('/content/drive/MyDrive/Emory/BMI NLP/Breast Cancer Data/CSV Files/roberta_prediction_1_only_with_sentiment.csv')
print("df original shape:", df.shape)

# Filter Tweets with negative Sentiments
df = df[df['sentiment_on_preprocessed']<=0.5].copy()
df = df.reset_index(drop=True)
print("df shape after only negative sentiments:", df.shape)

# Filter posts that contain treatment side-effect / negative keywords
trestment_bc_negative_keywords = ["treatment","medication","medicine","med","tablets","side effect","reaction","drug","tamoxifen","chemo","mental","emotion",
                     "lump","pain","sleep","docetaxel","oncologist","doc" "stop"]  # Keywords based on literature review of Pub_med papers (reference in main article)

treatment_regex = r'(?i)\b(?:' + '|'.join(trestment_bc_negative_keywords) + r')\b'
df_filtered = df[df['original_text'].str.contains(treatment_regex, na=False)].copy()
df_filtered = df_filtered.reset_index(drop=True)
print(df_filtered.shape)

df original shape: (154571, 5)
df shape after only negative sentiments: (108842, 5)
(13066, 5)


In [6]:
df_filtered

,text,original_text,roberta_predictions,sentiment_on_original,sentiment_on_preprocessed
0,hella breast cancer family ill 4th gen woman f...,i have hella breast cancer in my family (i'll ...,1,0.34210,0.40150
1,prestonjenny tonischelfhaut bit scare last yea...,@preston_jenny @toni_schelfhaut She had a bit ...,1,0.53220,0.34090
2,room rater trash dan bongino decor hotel room ...,Room Rater Trashes Dan Bongino For Decor in Ho...,1,0.17015,0.17015
3,bad carl zha wasnt around remember time mao ze...,Too bad Carl Zha wasn't around to remember the...,1,0.08200,0.08200
4,uksfwriter bbcnews itvnews skynews anecdotal e...,@uk_sf_writer @BBCNews @itvnews @SkyNews Anecd...,1,0.11280,0.06230
...,...,...,...,...,...
13061,lump throat,The lump on my throat 🙁,1,0.50000,0.50000
13062,witchnatural reaction exactly young talented n...,"@witch_natural My reaction exactly. So young, ...",1,0.15855,0.14520
13063,repboebert kamalaharris voted cancer treatment...,@RepBoebert @KamalaHarris You voted against ca...,1,0.01520,0.01630
13064,im entering sixth year colon cancer survivor 2...,I'm now entering my sixth year as a colon canc...,1,0.27980,0.26165


In [11]:
# Extracting tweets of self reported BC with negative sentiments
self_reported_texts = df_filtered['original_text'].to_list()
topic_model = BERTopic(verbose=True)
topics, probs = topic_model.fit_transform(self_reported_texts)

# Visualize Topic Modelling Figure
fig = topic_model.visualize_barchart(top_n_topics=50)
fig.show()


2024-03-31 03:51:25,513 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/409 [00:00<?, ?it/s]

2024-03-31 03:53:45,362 - BERTopic - Embedding - Completed ✓
2024-03-31 03:53:45,364 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-03-31 03:53:55,216 - BERTopic - Dimensionality - Completed ✓
2024-03-31 03:53:55,219 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-03-31 03:53:55,888 - BERTopic - Cluster - Completed ✓
2024-03-31 03:53:55,897 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-03-31 03:53:56,983 - BERTopic - Representation - Completed ✓


In [12]:
fig.show()